# Virtual Assistants and Accessing Data

### SQL basics
Time to begin writing queries for your first hotel booking chatbot! The database has been loaded as "hotels.db" and a cursor, which has access to the database, has already been defined for you as cursor.

Three queries are provided below. Your job is to identify which query returns ONLY the "Hotel California".

You can test each query below by calling the cursor's .execute() method and passing the query in as a string. Then, you can print the results by calling the cursor's .fetchall() method, which takes no arguments.

In [1]:
# Import sqlite3
import sqlite3

# Open connection to DB
conn = sqlite3.connect('hotels.db')

# Create a cursor
c = conn.cursor()

# Define area and price
area, price = "south", "hi"
t = (area, price)

# Execute the query
c.execute('SELECT * FROM hotels WHERE area=? AND price=?', t)

# Print the results
print(c.fetchall())

OperationalError: no such table: hotels

# Exploring a DB with Natural Language

### Creating queries from parameters
Now you're going to implement a more powerful function for querying the hotels database. The goal is for that function to take arguments that can later be specified by other parts of your code.

More specifically, your job is to define a find_hotels() function which takes a single argument - a dictionary of column names and values - and returns a list of matching hotels from the database.



In [2]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect('hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

### Using your custom function to find hotels
Here, you'll see your find_hotels() function in action! Recall that it accepts a single argument, params, which is a dictionary of column names and values.

In [3]:
# Create the dictionary of column names and values
params = {"area":"south","price":"lo"}

# Find the hotels that match the parameters
print(find_hotels(params))

OperationalError: no such table: hotels

### Creating SQL from natural language
Now you'll write a respond() function that can handle messages like "I want an expensive hotel in the south of town" and respond appropriately according to the number of matching results in a database. This is an important functionality for any database-backed chatbot.

Your find_hotels() function from the previous exercises has already been defined for you, along with a Rasa NLU interpreter object, which can handle hotel queries, and a list of responses, which you can explore in the Shell.

In [5]:
# Define respond()
def respond(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find hotels that match the dictionary
    results = find_hotels(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)

# Test the respond() function
respond("I want an expensive hotel in the south of town")

NameError: name 'interpreter' is not defined

# Incremental Slot Filling and Negation

### Refining your search
Now you'll write a bot that allows users to add filters incrementally, just in case they don't specify all of their preferences in one message.

To do this, initialize an empty dictionary params outside of your respond() function (as opposed to inside the function, like in the previous exercise). Your respond() function will take in this dictionary as an argument.

In [6]:
# Define a respond function, taking the message and existing params as input
def respond(message, params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

# Initialize params dictionary
params = {}

# Pass the messages to the bot
for message in ["I want an expensive hotel", "in the north of town"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))

USER: I want an expensive hotel


NameError: name 'interpreter' is not defined

### Basic negation
Quite often, you'll find your users telling you what they don't want - and that's important to understand! In general, negation is a difficult problem in NLP. Here, we'll take a very simple approach that works for many cases.

A list of tests called tests has been defined for you. Explore it in the Shell - you'll find that each test is a tuple consisting of:

A string containing a message with entities.
A dictionary containing the entities as keys and a Boolean saying whether they are negated as the key.
Your job is to define a function called negated_ents() which looks for negated entities in a message.

In [7]:
# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity contains a negation, assign the key to be False
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(negated_ents(test[0]) == test[1])

NameError: name 'tests' is not defined

### Filtering with excluded slots
Now you're going to put together some of the ideas from previous exercises in order to allow users to tell your bot about what they do and do not want, split across multiple messages.

The negated_ents() function has already been defined for you. Additionally, a slightly tweaked version of the find_hotels() function, which accepts a neg_params dictionary in addition to a params dictionary, has been defined.

In [8]:
# Define the respond function
def respond(message,params,neg_params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    ent_vals = [e["value"] for e in entities]
    # Look for negated entities
    negated = negated_ents(message,ent_vals)
    for ent in entities:
        if ent["value"] in negated and negated[ent["value"]]:
            neg_params[ent["entity"]] = str(ent["value"])
        else:
            params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_hotels(params,neg_params)
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Return the correct response
    return responses[n].format(*names), params, neg_params

# Initialize params and neg_params
params = {}
neg_params = {}

# Pass the messages to the bot
for message in ["I want a cheap hotel", "but not in the north of town"]:
    print("USER: {}".format(message))
    response, params, neg_params = respond(message, params, neg_params)
    print("BOT: {}".format(response))

USER: I want a cheap hotel


NameError: name 'interpreter' is not defined